In [1]:
import os
import sys
sys.path.insert(0, './../')
import bbdc2021 as bbdc

In [1]:
pipe_param = {'data_folder': './../data/',
                  'wav_files_folder': 'final_pre_dataset', # mel parameter
                  'frame_size': 0.04, 'frame_stride': 0.02495,
                  'mel_filter': 32, 'nfft': 512,
                  'dev_csv': 'dev-labels.csv', # filelisten, falls man doch auf kleinerem probieren moechte
                  'eval_csv': 'challenge_filelist_dummy.csv',
                  'scaling': 'standard', # 'standard', 'minmax' oder 'no' wenn kein scaling sein soll
                  'test_split_range': (8000, -1), # index anfang und ende aus dev set 
                  'prediction_path': './', # wo prediction csvs gespeichert werden sollen
                  'post_processing': 'fill', # 'fill' is das von Jannes.
                  'post_timethres': 0.5, # post processing parameter
                  'post_noisethres': 0.3,
                  'post_base': 2,
                  'submission_file_path': './challenge_submission.csv'}
unet_param = {'channels': [32, 64, 64],
              'val_split_range': (6400, -1), # 0.2
              'loss': 'categorical_crossentropy', # bbdc.dice_loss moeglich
              'learning_rate': 0.001,
              'batch_size': 20,
              'epochs': 30,
              'model_save_path': './',
              'load_path': None} # Wenn trainiert werden soll: None; Submission 1: 'model9561.h5'

In [2]:
def pipeline_u_net2d_1(pipe_param, model_param):
    """Current u_net2d pipeline from loading data to prediction.
    Takes paramater dictionary as argument"""
    print('Pipeline parameter:', pipe_param)
    print('Model parameter:', model_param)
    # load
    x_dev, y_dev, timep, filelist_dev, x_ch, filelist_ch = bbdc.loading_block2(pipe_param)
    # shuffle
    x_dev, y_dev, filelist_dev = bbdc.shuffle_block1(x_dev, y_dev, filelist_dev)
    # split (tv=train and val set)
    x_tv, x_test, y_tv, y_test, filelist_test = bbdc.split_block1(x_dev, y_dev, timep, filelist_dev, pipe_param)
    # model fit (or load)
    history, model = bbdc.model_block1_unet2d(x_tv, y_tv, model_param)
    # evaluate
    scores_list, model_psds = bbdc.evaluation_block1(x_test, y_test, timep, filelist_test, model, pipe_param)
    # post processing and its evaluation
    pp_pred_test, pp_pred_ch, pp_psds = bbdc.postprocessing_with_evaluation_block1(x_test, y_test, timep,
                                                                                   filelist_test, x_ch,
                                                                                   filelist_ch, model, pipe_param)
    # challenge prediction
    bbdc.challenge_prediction_block1(pp_pred_ch, timep, filelist_ch, pipe_param)
    return scores_list, model_psds, pp_psds

In [3]:
pipeline_u_net2d_1(pipe_param, unet_param)

Pipeline parameter: {'data_folder': './../data/', 'wav_files_folder': 'final_pre_dataset', 'frame_size': 0.04, 'frame_stride': 0.02495, 'mel_filter': 32, 'nfft': 512, 'dev_csv': 'dev-labels.csv', 'eval_csv': 'challenge_filelist_dummy.csv', 'scaling': 'standard', 'test_split_range': (8000, -1), 'prediction_path': './', 'post_processing': 'fill', 'post_timethres': 0.5, 'post_noisethres': 0.3, 'post_base': 2, 'submission_file_path': './challenge_submission.csv'}
Model parameter: {'channels': [32, 64, 64], 'val_split_range': (6400, -1), 'loss': 'categorical_crossentropy', 'learning_rate': 0.001, 'batch_size': 20, 'epochs': 30, 'model_save_path': './', 'load_path': None}


NameError: name 'bbdc' is not defined

In [ ]:
# see colab notebook unet for 2dunet calculations